In [1]:
# Load chat
import pandas as pd
import numpy as np
import re
import sqlalchemy
from whatstk import WhatsAppChat
from mysql.connector import connect
from sqlalchemy import create_engine
filepath = 'SE_chat.txt'
chat = WhatsAppChat.from_source(filepath=filepath)
chat_df=chat.df
members_name=pd.read_excel('SE_Chat.xlsx', index_col=None, sheet_name='members')

database_username = 'python_access'
database_password = 'password'
database_ip       = 'localhost'
database_name     = 'se_observation_log'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))
observable_object=pd.read_sql('SELECT * from observable_objects', database_connection)



In [2]:
chat_df['obv_date'] = pd.to_datetime(chat_df['date']).dt.date
chat_df['obv_time'] = pd.to_datetime(chat_df['date']).dt.time
chat_df=chat_df.drop(['date'],axis=1)
chat_df.tail()
chat_df['contains_observation']=''
chat_df['observation']=''
chat_df['obv_date'] = pd.to_datetime(chat_df['obv_date'], format='%Y-%m-%d')
observation='|'.join(observable_object['object'])
chat_df['contains_observation']=chat_df.message.str.contains(observation, flags=re.IGNORECASE, regex=True)
chat_df

,username,message,obv_date,obv_time,contains_observation,observation
0,Frances Cahill,IMG-20230720-WA0005.jpg (file attached),2023-07-19,23:57:00,False,
1,Frances Cahill,IMG-20230720-WA0004.jpg (file attached),2023-07-19,23:58:00,False,
2,Frances Cahill,IMG-20230720-WA0003.jpg (file attached),2023-07-19,23:58:00,False,
3,Frances Cahill,IMG-20230720-WA0002.jpg (file attached),2023-07-19,23:58:00,False,
4,Frances Cahill,IMG-20230720-WA0002.jpg (file attached),2023-07-19,23:58:00,False,
...,...,...,...,...,...,...
422,John hackett,Great photos 🌝,2023-08-31,23:57:00,False,
423,Joe Howlett,📡🔭🚀🛰️👍,2023-09-01,05:25:00,False,
424,Cathal O'D,You deleted this message,2023-09-01,07:10:00,False,
425,Cathal O'D,Looks like we'll have a clear sky tonight. We ...,2023-09-01,07:12:00,True,


In [3]:
def get_keywords(row):
    some_text = row['message']
    tokens = observation
    keywords = [keyword for keyword in tokens if keyword.isalpha()]
    keywords_string = ','.join(keywords)
    return keywords_string
# chat_df['observation'] = chat_df.apply(get_keywords, axis=1)

In [4]:
this_month=chat_df.loc[(chat_df['obv_date']>='2023-08-01') & (chat_df['obv_date']<='2023-08-31')]
# with pd.ExcelWriter('observation_log.xlsx', engine='xlsxwriter', datetime_format= "dd-mm-yy") as writer:
    #this_month.to_excel(writer)
this_month.to_csv('observation_log.csv')
this_month

,username,message,obv_date,obv_time,contains_observation,observation
93,Frances Cahill,IMG-20230801-WA0002.jpg (file attached) Fox on...,2023-08-01,00:06:00,False,
94,Frances Cahill,IMG-20230801-WA0000.jpg (file attached) Beauti...,2023-08-01,00:06:00,True,
95,Frances Cahill,IMG-20230801-WA0001.jpg (file attached),2023-08-01,00:07:00,False,
96,Emily,Oh wow!!! How lovely😊,2023-08-01,07:58:00,False,
97,Joe Howlett,📡🔭🚀🛰️👍,2023-08-01,08:38:00,False,
...,...,...,...,...,...,...
418,Frances Cahill,Great we got to see the Superblue Moon..rising,2023-08-31,22:11:00,True,
419,Helen Byrne,"Well done, everyone. 👍",2023-08-31,23:25:00,False,
420,Emily,Oh gorgeous pictures,2023-08-31,23:35:00,False,
421,John hackett,Cloud & still raining here 😐 but great so many...,2023-08-31,23:56:00,True,
